# Import Packages

In [17]:
import sys
 
import os
import pandas as pd
import numpy as np
import pickle
import SFR_PD_Recalibration_2023_Lib as tool
# import Concentra_SFR_Fit_for_Use_2024_Lib as lib
from matplotlib import pyplot as plt

import Lib_RWA as Lib_RWA

# delete lib_RWA from system modules 

del(sys.modules['Lib_RWA'])

# delete SFR_PD_Recalibration_2023_Lib from system modules

del(sys.modules['SFR_PD_Recalibration_2023_Lib'])


import SFR_PD_Recalibration_2023_Lib as tool

import Lib_RWA as Lib_RWA

import math

In [18]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Inputs

In [19]:

# Snaphsot and RWA file information

# snapshot_num = 202212 # 20250403 Fran commented out as George in S1
# snapshot = str(snapshot_num) # 20250403 Fran commented out as George in S1
# snapshot_date = '2022-12-31' # 20250403 Fran commented out as George in S1
# RWA_File_from_Chenxi = 'SFR_RWA_202212.csv' #20250403 Fran added as George S1!!! Note to be consistent with the snapshot date

snapshot_num = 202412 # 20250403 Fran added as George in S1 
snapshot = str(snapshot_num) # 20250403 Fran added as George in S1
snapshot_date = '2024-12-31' # 20250403 Fran added as George in S1
RWA_File_from_Chenxi = 'SFR_202412_v2 (from Chenxi 20250403).csv' #20250403 Fran added as George S1!!! Note to be consistent with the snapshot date

In [20]:
#CAR 2023 Ch5 prescribed parameters

correlation_residential_mortgages = 0.15
correlation_residential_mortgages_rental = 0.22

CMHC_pd = 0.0001
CMHC_lgd = 0.11 #according to the newly developed Sovereign LGD model

lgd_gen_floor = 0.1 #general floor by CAR 2023

In [21]:
# Production Server and Tables

Server_Finance = 'EQDWP01'  
Database_Production = 'ET_Finance_Production' 
Server_Consolidated  =  'EQSQLT01\RISK'
Database_Consolidated = 'Risk_Analytics'


In [91]:
# Columns to Keep

RWA_Info_Cols_to_Keep = ['Loan_Number','Insured_class','EAD','Advance_Amount','Years_to_maturity', 'corr_uninsured', 'RWA_standardized']

PD_Cols_to_Keep = ['Loan_Number','SL_Date','PD_Pre_MOC','PD_Post_MOC_Pre_Adj','PD_Post_MOC','MRS_Bin_PD','Insured_Ind','Alt_Prime_Indicator','RemainingPrincipal_Tot_Exp','RemainingPrincipal_Excl_Partner', 'Corporate_Applicant_Ind','Combo_Province_Metro_Override_WOE', 'Prior_24_Worse_Delinquent_Status_FMT_Adj', 'Prior_24_Worse_Delinquent_Status_FMT_Adj_WOE','Beacon_Avg_App_CoApp','Beacon_Avg_App_CoApp_WOE','BF_LTV_Tot_Exp_FSA_Dw_WF','Combo_LTV_Insured_Ind_WOE']

PD_Cols_for_Corp = ['AdvancedAmount_EQB_Exp','AdvancedAmount_Total_Exp','AdvancedAmt_Incl_Part','AdvancedAmt_Excl_Part','Remaining_Term'] 


LGD_Cols_to_Keep = ['Loan_Number','MRS_Bin_LGD','Model_LGD','Mapped_LGD','Appraisal_Bin_WOE', 'Occupancy_WOE', 'Province_Foreclosure_WOE','LTV_Bin_WOE', 'Segment_Avg_LGD','Base_Line_LGD','LGD_DT_Adjusted']



# Set Folder

In [23]:
# Get current working directory

current_dir = os.getcwd()  # 20250403 George added

# code_dir= "C:\\Users" +'\\' + username + '\\' + "Equitable Bank\\EQB-Concentra Fit for Use - Fit_for_Use Development - Fit_for_Use Development\\RWA\\code" # 20250403 George commented out Francesca/Joseph's code

code_dir = current_dir # 20250403 George added


if int(snapshot_num) == 202412:
    input_dir = code_dir + '\\..\\' + "Inputs" # 20250406 George added
    Output_dir = code_dir + '\\..\\' + "Outputs" #20250406 George added

if int(snapshot_num) == 202212:
    input_dir = code_dir + '\\..\\' + "Dec. 2022 RWA Inputs" #20250410 George added
    Output_dir = code_dir + '\\..\\' + "Replicated Dec 2022 Outputs" #20250410 George added




# Run ID for Production Tables

In [24]:

def sql_max_FeedID( snapshot ):
    
    table = 'ET_Finance_Production.dbo.tb_RE_log' # 20250410 George added

    if pd.to_datetime(snapshot) <= pd.to_datetime('2024-10-31'):
        table = table + '_G2'  # 20250410 George added for Generation 2 model

    sql = f'''
      SELECT max([FeedID]) as max_FeedID
       FROM {table}
       where Reporting_date =  {snapshot} 
             
            '''  
    return sql

Max_Feed_ID= tool.download_from_sql( Server_Finance, Database_Production, sql_max_FeedID( "'" + snapshot_date + "'" ) )['max_FeedID'][0]


c:\Users\georgez\OneDrive - Equitable Bank\MD_RWA_Impact_Analysis (Joseph)\SFR\code\SFR_PD_Recalibration_2023_Lib.py:1732: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn,**kwargs)


# Load RWA Data Info

In [25]:

# Read in the RWA data from Chenxi

# Import data with Loan_Number,Insured_class,CalibratedPD,Final_LGD,EAD,Advance_Amount,Years_to_maturity and corr_uninsured
##################### load snapshot data Chenxi provided and rename#################################
#rwa_data = pd.read_csv(input_dir +'\\'+'SFR_RWA_' + snapshot + '.csv', low_memory=False) #20250403 Fran commented as George in S1
rwa_data = pd.read_csv(input_dir +'\\'+ RWA_File_from_Chenxi, low_memory=False) #20250403 Fran added as George in S1

rwa_raw_data_0 = rwa_data[RWA_Info_Cols_to_Keep]  # 20250410 George add

In [26]:
rwa_data.shape

(53402, 17)

In [27]:
#Floor the EAD on 0          

rwa_raw_data_0.loc[rwa_raw_data_0['EAD']<0, ['EAD']] =0


# Load Consolidated RESL Data

## Extract from SQL

In [28]:
# retrieve risk rating attributes from Production

def sql_consolidated_RESL(snapshot_date):
    
    table = '[Risk_Analytics].[dbo].Risk_Consolidated_RESL' 

    sql = f'''
    Select
        [Year]
      ,[Time]
      ,[SL_Date]
      ,[Company]
      ,[Loan_Number]
      ,[CIF_number]
      ,[Business_Line]
      ,[Loan_Status]
      ,[Loan_Type]
      ,[Property_Code]
      ,[Address]
      ,[City]
      ,[Province]
      ,[Postal_Code]
      ,[Metro]
      ,[Tier]
      ,[Securitized_Indicator]
      ,[Alt_Prime_Ind]
      ,[Insured_Ind]
      ,[Insurer_Name]
      ,[Standard_Risk_Rating]
      ,[EQB_Authorized_Loan_Amount]
      ,[Remaining_Principal]
      ,[Original_Principal_Amount]
      ,[HELOC_Balance]
      ,[Accrued_Interest]
      ,[Sundry_Account]
      ,[Fair_Value]
      ,[Exposure_at_Default]
      ,[Borrower_Name]
      ,[Date_Of_Birth]
      ,[Borrower_Age]
      ,[Servicer_ID]
      ,[Servicer_Name]
      ,[Servicer_Name_Full]
      ,[Maturity_Date]
      ,[Renewal_Date]
      ,[Advance_Date]
      ,[Funded_Date]
      ,[Appraisal_Value_Inception]
      ,[Appraisal_Value_Current]
      ,[LTV_Inception]
      ,[LTV_Current]
      ,[Remaining_Term]
      ,[Term]
      ,[Interest_Rate_Raw]
      ,[Interest_Rate_New]
      ,[Rate_Type]
      ,[Payment_Plan]
      ,[Payment_Plan_Description]
      ,[Default_Indicator]
      ,[Arrears_Days]
      ,[Arrears_Status]
      ,[Arrears_Days_Group]
      ,[Gemstone]
      ,[Inception_Beacon_Score]
      ,[Current_Beacon_Score]
      ,[Current_BNI_Score]
      ,[Amortization_Inception]
      ,[Amort_Rem]
      ,[Branch]
      ,[Occupancy]
      ,[Funding_GDS]
      ,[Funding_TDS]
      ,[Conforming_Ind_SFRAlt]
      ,[Conforming_Reason]
      ,[Dwelling_Type]
      ,[Salary_BFS_Indicator]
      ,[Total_Household_Income]
      ,[Loan_Purpose]
      ,[Third_Party_Name]
      ,[High_Rise_Condo_Indicator]
      ,[AIRB_Retail_Model_PD_Value]
      ,[AIRB_Retail_Calibrated_PD_Value]
      ,[AIRB_Retail_PD_Pool]
      ,[AIRB_Retail_Model_LGD_Value]
      ,[AIRB_Retail_Calibrated_LGD_Value]
      ,[AIRB_Retail_Downturn_LGD_Value]
      ,[AIRB_Retail_Final_LGD_Value]
      ,[AIRB_Retail_LGD_Pool]
      ,[LOC_Credit_Limit]
      ,[Youngest_Date_Of_Birth]
      ,[Youngest_Age]
      ,[Last_Update_Date]
      ,[Last_Update_By]
      from [Risk_Analytics].[dbo].[Risk_Consolidated_RESL]
      where SL_Date = '{snapshot_date}' and Company = 'EQB'
            '''  
    return sql


Consolidated_RESL_Data_Raw = tool.download_from_sql(Server_Consolidated, Database_Consolidated, sql_consolidated_RESL(snapshot_date))

c:\Users\georgez\OneDrive - Equitable Bank\MD_RWA_Impact_Analysis (Joseph)\SFR\code\SFR_PD_Recalibration_2023_Lib.py:1732: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn,**kwargs)


In [29]:
# Count values of Third_Party_Name

Consolidated_RESL_Data_Raw['Third_Party_Name'].value_counts()

EQB               51174
First National     7162
Paradigm           3034
Name: Third_Party_Name, dtype: int64

## Clean Data

In [30]:
# Change Loan_Number to integer

Consolidated_RESL_Data_Raw['Loan_Number'] = Consolidated_RESL_Data_Raw['Loan_Number'].astype(int)

In [31]:
# Keep columns Loan_Number, SL_Date, Alt_Prime_Ind, Insured_Ind, Default_Indicator, Third_Party_Name

Consolidated_RESL_Data = Consolidated_RESL_Data_Raw[['Loan_Number','Alt_Prime_Ind','Default_Indicator','Third_Party_Name', 'SL_Date']]

# Rename columns: Alt_Prime_Ind to Alt_Prime_Indicator, Third_Party_Name to Sub_Product 

Consolidated_RESL_Data.rename(columns={'Alt_Prime_Ind':'Alt_Prime_Indicator', 'Third_Party_Name':'Sub_Product'}, inplace=True)


C:\Users\georgez\AppData\Local\Temp\ipykernel_8452\160977798.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Consolidated_RESL_Data.rename(columns={'Alt_Prime_Ind':'Alt_Prime_Indicator', 'Third_Party_Name':'Sub_Product'}, inplace=True)


# Load Risk Rating Attributes

## Extract Risk Rating Attributes from SQL

In [32]:
# retrieve risk rating attributes from Production

def sql_risk_rating_attributes(Year,Time):
    
    table = '[ET_Finance_Production].[dbo].t_SFR_Risk_Rating_Attributes' # 20250410 George added

    if pd.to_datetime(snapshot_date) <= pd.to_datetime('2024-10-31'):
        table = table + '_G2'  # 20250410 George added for Generation 2 model

    sql = f'''
      SELECT 
       [Year]
      ,[Time]
      ,[Loan_Number]
      ,[X1_Average_Beacon_Score_PD]
      ,[X2_Current_LTV_PD]
      ,[X2_Loan_Class_PD]
      ,[X3_Worst_Delinquency_Two_Years_Max_PD]
      ,[X4_Province_PD]
      ,[X1_FSA_LGD]
      ,[X1_Province_LGD]
      ,[X2_LTV_LGD]
      ,[X3_Appraisal_LGD]
      ,[X4_Occupancy_LGD]
      ,[Current_Delinquency_Days]
      ,[Unlikely_to_Pay]
      ,[Remaining_Principal_EQB]
      ,[Remaining_Principal_with_Partner]
      ,[Remaining_Principal_with_Partner_HELOC]
      ,[DEF_Current_Balance_HELOC]
      ,[Partner_Percentage]
      ,[Funding_Appraisal_Value]
      ,[Purchase_Price]
      ,[Property_Value]
      ,[Tenure]
      ,[Risk_Rating_Tier]
      ,[FSA]
      ,[City]
      ,[Metro]
      ,[Province]
      ,[Dwelling_Type]
      ,[Fair_Values]
      ,[Accrued_Interest]
      ,[Advance_Date]
      ,[Renewal_Date]
      ,[old_Loan_Number_Secondary_Suites]
      ,[HPI_Funding_FSA_Dwelling]
      ,[HPI_Funding_FSA_Dwelling_Style]
      ,[HPI_Funding_FSA_Aggregate]
      ,[HPI_Reporting_FSA_Dwelling]
      ,[HPI_Reporting_FSA_Dwelling_Style]
      ,[HPI_Reporting_FSA_Aggregate]
      ,[HPI_Funding_City_Dwelling]
      ,[HPI_Funding_City_Dwelling_Style]
      ,[HPI_Funding_City_Aggregate]
      ,[HPI_Reporting_City_Dwelling]
      ,[HPI_Reporting_City_Dwelling_Style]
      ,[HPI_Reporting_City_Aggregate]
      ,[HPI_Funding_Metro_Dwelling]
      ,[HPI_Funding_Metro_Dwelling_Style]
      ,[HPI_Funding_Metro_Aggregate]
      ,[HPI_Reporting_Metro_Dwelling]
      ,[HPI_Reporting_Metro_Dwelling_Style]
      ,[HPI_Reporting_Metro_Aggregate]
      ,[HPI_Funding_Province_Dwelling]
      ,[HPI_Funding_Province_Dwelling_Style]
      ,[HPI_Funding_Province_Aggregate]
      ,[HPI_Reporting_Province_Dwelling]
      ,[HPI_Reporting_Province_Dwelling_Style]
      ,[HPI_Reporting_Province_Aggregate]
      ,[HPI_Funding_National_Dwelling]
      ,[HPI_Funding_National_Dwelling_Style]
      ,[HPI_Funding_National_Aggregate]
      ,[HPI_Reporting_National_Dwelling]
      ,[HPI_Reporting_National_Dwelling_Style]
      ,[HPI_Reporting_National_Aggregate]
      ,[DLGD_BF_LTV]
      ,[EAD]
      ,[Remaining_Principal_Post_CRM]
      ,[Principal_Amount_Post_CRM]
      ,[Loan_Class_Post_CRM]
      ,[Price_Correction]
      ,[Teranet_Region]
      ,[Teranet_Funding_HPI]
      ,[Teranet_Reporting_HPI]       FROM {table}
       where Year =  '{Year}' and Time = '{Time}'
            '''  
    return sql


Risk_Rating_Attribute_Prod_Data_Raw = tool.download_from_sql( Server_Finance, Database_Production, sql_risk_rating_attributes(Year='Y'+ str(pd.to_datetime(snapshot_date).year), Time=pd.to_datetime(snapshot_date).strftime('%b') ))

c:\Users\georgez\OneDrive - Equitable Bank\MD_RWA_Impact_Analysis (Joseph)\SFR\code\SFR_PD_Recalibration_2023_Lib.py:1732: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn,**kwargs)


## Sanity Check

In [33]:
 # Print columns in alphabetical order

print(Risk_Rating_Attribute_Prod_Data_Raw.columns.sort_values())


Index(['Accrued_Interest', 'Advance_Date', 'City', 'Current_Delinquency_Days', 'DEF_Current_Balance_HELOC', 'DLGD_BF_LTV', 'Dwelling_Type', 'EAD', 'FSA', 'Fair_Values', 'Funding_Appraisal_Value', 'HPI_Funding_City_Aggregate', 'HPI_Funding_City_Dwelling', 'HPI_Funding_City_Dwelling_Style', 'HPI_Funding_FSA_Aggregate', 'HPI_Funding_FSA_Dwelling', 'HPI_Funding_FSA_Dwelling_Style', 'HPI_Funding_Metro_Aggregate', 'HPI_Funding_Metro_Dwelling', 'HPI_Funding_Metro_Dwelling_Style', 'HPI_Funding_National_Aggregate', 'HPI_Funding_National_Dwelling', 'HPI_Funding_National_Dwelling_Style', 'HPI_Funding_Province_Aggregate', 'HPI_Funding_Province_Dwelling', 'HPI_Funding_Province_Dwelling_Style', 'HPI_Reporting_City_Aggregate', 'HPI_Reporting_City_Dwelling', 'HPI_Reporting_City_Dwelling_Style', 'HPI_Reporting_FSA_Aggregate', 'HPI_Reporting_FSA_Dwelling', 'HPI_Reporting_FSA_Dwelling_Style', 'HPI_Reporting_Metro_Aggregate', 'HPI_Reporting_Metro_Dwelling', 'HPI_Reporting_Metro_Dwelling_Style',
       'HP

## Data Cleaning

In [34]:
# Keep Columns Loan_Number, Remaining_Principal_EQB, Remaining_Principal_with_Partner, Remaining_Principal_with_Partner_HELOC, Remaining_Principal_Post_CRM, 'X1_Average_Beacon_Score_PD', 'X2_Current_LTV_PD', 'X2_Loan_Class_PD', 'X3_Worst_Delinquency_Two_Years_Max_PD', 'X4_Province_PD', 'X1_FSA_LGD', 'X1_Province_LGD', 'X2_LTV_LGD', 'X3_Appraisal_LGD', 'X4_Occupancy_LGD', 'Current_Delinquency_Days'

Risk_Rating_Attribute_Prod_Data = Risk_Rating_Attribute_Prod_Data_Raw[['Loan_Number','Remaining_Principal_EQB','Remaining_Principal_with_Partner_HELOC','X1_Average_Beacon_Score_PD','X2_Current_LTV_PD','X2_Loan_Class_PD','X3_Worst_Delinquency_Two_Years_Max_PD','X2_LTV_LGD','X3_Appraisal_LGD','X4_Occupancy_LGD','Current_Delinquency_Days']] 

# Renaming columns: Remaining_Principal_EQB to RemainingPrincipal_Excl_Partner, Remainining_Principal_with_Partner_HELOC to RemainingPrincipal_Tot_Exp, X1_Average_Beacon_Score_PD to Beacon_Avg_App_CoApp, X2_Current_LTV_PD to BF_LTV_Tot_Exp_FSA_Dw_WF, X2_Loan_Class_PD to Insured_Ind, X3_Worst_Delinquency_Two_Years_Max_PD to Prior_24_Worse_Delinquent_Status_FMT_Adj, X2_LTV_LGD to BF_LTV_Incl_Parter_Incl_HELOC_FSA_Dw, X3_Appraisal_LGD to BF_Appr_Prov_Dw, X4_Occupancy_LGD

Risk_Rating_Attribute_Prod_Data.rename(columns={'Remaining_Principal_EQB':'RemainingPrincipal_Excl_Partner', 'Remaining_Principal_with_Partner_HELOC':'RemainingPrincipal_Tot_Exp', 'X1_Average_Beacon_Score_PD':'Beacon_Avg_App_CoApp', 'X2_Current_LTV_PD':'BF_LTV_Tot_Exp_FSA_Dw_WF', 'X2_Loan_Class_PD':'Insured_Ind', 'X3_Worst_Delinquency_Two_Years_Max_PD':'Prior_24_Worse_Delinquent_Status_FMT_Adj', 'X2_LTV_LGD':'BF_LTV_Incl_Parter_Incl_HELOC_FSA_Dw', 'X3_Appraisal_LGD':'BF_Appr_Prov_Dw', 'X4_Occupancy_LGD':'Occupancy'}, inplace=True) 

# Change the values of Insured_Ind: Uninsured to 'Not Insured'

Risk_Rating_Attribute_Prod_Data['Insured_Ind'] = Risk_Rating_Attribute_Prod_Data['Insured_Ind'].replace({'Uninsured':'Not Insured'})


C:\Users\georgez\AppData\Local\Temp\ipykernel_8452\1469802393.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Risk_Rating_Attribute_Prod_Data.rename(columns={'Remaining_Principal_EQB':'RemainingPrincipal_Excl_Partner', 'Remaining_Principal_with_Partner_HELOC':'RemainingPrincipal_Tot_Exp', 'X1_Average_Beacon_Score_PD':'Beacon_Avg_App_CoApp', 'X2_Current_LTV_PD':'BF_LTV_Tot_Exp_FSA_Dw_WF', 'X2_Loan_Class_PD':'Insured_Ind', 'X3_Worst_Delinquency_Two_Years_Max_PD':'Prior_24_Worse_Delinquent_Status_FMT_Adj', 'X2_LTV_LGD':'BF_LTV_Incl_Parter_Incl_HELOC_FSA_Dw', 'X3_Appraisal_LGD':'BF_Appr_Prov_Dw', 'X4_Occupancy_LGD':'Occupancy'}, inplace=True)
C:\Users\georgez\AppData\Local\Temp\ipykernel_8452\1469802393.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

In [35]:
# Change Loan_Number to integer

Risk_Rating_Attribute_Prod_Data['Loan_Number'] = Risk_Rating_Attribute_Prod_Data['Loan_Number'].astype(int)

C:\Users\georgez\AppData\Local\Temp\ipykernel_8452\937766065.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Risk_Rating_Attribute_Prod_Data['Loan_Number'] = Risk_Rating_Attribute_Prod_Data['Loan_Number'].astype(int)


# Load PD Driver Data

## Extract from SQL

In [36]:
# retrieve PD WOE data from Production

def sql_PD_Driver(snapshot_date):
    
    table = '[Risk_Modelling_Dataset].[dbo].[vw_SFR_PD_Model_Driver_Dataset_Including_Post_Def]'

    sql = f'''
      SELECT 
      [Loan_Number],
      Sub_Product,
      Default_Ind,
      Post_Default_Ind
       FROM {table}
       where SL_Date =  '{snapshot_date}'
            '''  
    return sql

PD_Driver_Data_Raw = tool.download_from_sql(server = Server_Consolidated, database = 'Risk_Modelling_Dataset', sql = sql_PD_Driver(snapshot_date) )


c:\Users\georgez\OneDrive - Equitable Bank\MD_RWA_Impact_Analysis (Joseph)\SFR\code\SFR_PD_Recalibration_2023_Lib.py:1732: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn,**kwargs)


## Clean Data

In [37]:
# Change Loan_Number to integer

PD_Driver_Data_Raw['Loan_Number'] = PD_Driver_Data_Raw['Loan_Number'].astype(int)

# Load PD WOE Data

## Extract PD WOE from SQL

In [38]:
# retrieve PD WOE data from Production

def sql_PD_WOE( Max_Feed_ID ):
    
    table = 'ET_Finance_Production.dbo.tb_X_WoE_PD_Result' # 20250410 George added

    if pd.to_datetime(snapshot_date) <= pd.to_datetime('2024-10-31'):
        table = table + '_G2'  # 20250410 George added for Generation 2 model

    sql = f'''
      SELECT 
      [LoanNumber]
      ,[X1_Average_Beacon_WoE]
      ,[X2_Current_LTV_WoE]
      ,[X3_Worst_Delinquency_Status_WoE]
      ,[X4_Province_WoE]
      ,[LogOdds]
      ,[CalibratedPD]
      ,[PDResult]
      ,[RiskRating]
      ,[RiskRatingCategory]
       FROM {table}
       where RunID =  {Max_Feed_ID} 
            '''  
    return sql

PD_WOE_Prod_Data_Raw = tool.download_from_sql( Server_Finance, Database_Production, sql_PD_WOE(Max_Feed_ID) )



c:\Users\georgez\OneDrive - Equitable Bank\MD_RWA_Impact_Analysis (Joseph)\SFR\code\SFR_PD_Recalibration_2023_Lib.py:1732: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn,**kwargs)


## Sanity Check

In [39]:
# Shape of PD_WOE_Prod_Data_Raw

print('Shape of PD_WOE_Prod_Data_Raw:')
print(PD_WOE_Prod_Data_Raw.shape)


# Group by RiskRating, count the number of loans
print('Count of loans by RiskRating:')
PD_WOE_Prod_Data_Raw['RiskRating'].value_counts()
# Group by RiskRating, get min and max of CalibratedPD

print('Min and max of CalibratedPD by RiskRating:')

PD_WOE_Prod_Data_Raw.groupby(['RiskRating']).agg({'CalibratedPD': ['min', 'max']}).reset_index().sort_values(by=['RiskRating'])

# Count values of X2_Current_LTV_WoE

print('Count of values of X2_Current_LTV_WoE:')

PD_WOE_Prod_Data_Raw['X2_Current_LTV_WoE'].value_counts()

# Count value of X4_Province_WoE

print('Count of values of X4_Province_WoE:')

PD_WOE_Prod_Data_Raw['X4_Province_WoE'].value_counts()

Shape of PD_WOE_Prod_Data_Raw:
(53402, 10)
Count of loans by RiskRating:
Min and max of CalibratedPD by RiskRating:
Count of values of X2_Current_LTV_WoE:
Count of values of X4_Province_WoE:


 0.165650    26175
-0.389936    13575
-0.161658     7464
 0.253031     5448
 0.000000      740
Name: X4_Province_WoE, dtype: int64

## Data Cleaning

In [40]:
# Rename columns in PD_WOE_Prod_Data_Raw: CalibratedPD to PD_Post_MOC, RiskRating to MRS_Bin_PD, X1_Average_Beacon_WoE to Beacon_Avg_App_CoApp_WOE, X2_Current_LTV_WoE to Combo_LTV_Insured_Ind_WOE, X3_Worst_Delinquency_Status_WoE to Prior_24_Worse_Delinquent_Status_FMT_Adj_WOE, X4_Province_WoE to Combo_Province_Metro_Override_WOE, LoanNumber to Loan_Number

PD_WOE_Prod_Data = PD_WOE_Prod_Data_Raw.rename(columns={'CalibratedPD': 'PD_Post_MOC', 'RiskRating': 'MRS_Bin_PD', 'X1_Average_Beacon_WoE': 'Beacon_Avg_App_CoApp_WOE', 'X2_Current_LTV_WoE': 'Combo_LTV_Insured_Ind_WOE', 'X3_Worst_Delinquency_Status_WoE': 'Prior_24_Worse_Delinquent_Status_FMT_Adj_WOE', 'X4_Province_WoE': 'Combo_Province_Metro_Override_WOE', 'LoanNumber': 'Loan_Number'})

# Change Loan_Number to integer

PD_WOE_Prod_Data['Loan_Number'] = PD_WOE_Prod_Data['Loan_Number'].astype(int)


In [41]:
# Keep Columns Loan_Number, PD_Post_MOC, MRS_Bin_PD, Beacon_Avg_App_CoApp_WOE, Combo_LTV_Insured_Ind_WOE, Combo_Province_Metro_Override_WOE, Prior_24_Worse_Delinquent_Status_FMT_Adj_WOE

PD_WOE_Prod_Data = PD_WOE_Prod_Data[['Loan_Number','PD_Post_MOC','MRS_Bin_PD','Beacon_Avg_App_CoApp_WOE','Combo_LTV_Insured_Ind_WOE','Combo_Province_Metro_Override_WOE','Prior_24_Worse_Delinquent_Status_FMT_Adj_WOE']]


# Load PD Scoring Data

## Extract Raw Data

In [42]:
# add new SFR PD model generated PD 

#new_pd_data = pd.read_csv( Active_dir +'\\'+'PARAM_Pred_PD'  + '.csv', low_memory=False) # 20250402 George Commented out 
PD_Data_Scored = pd.read_pickle(input_dir + '\\Rating_July2020_to_Dec2024_EQB (from George 20250414).pkl') # 20250402 George added

PD_Data_Scored_red = PD_Data_Scored.loc[PD_Data_Scored['SL_Date'] == snapshot_date]

## Compare with PD Driver Data

In [43]:
# Merge PD_Data_Scored_red with PD_Driver_Data_Raw on Loan_Number

PD_Scoring_Driver_Merge = pd.merge(PD_Data_Scored_red, PD_Driver_Data_Raw[['Loan_Number']], on='Loan_Number', how='outer', indicator= 'Merge_PDScoring_PDDriver')

# Change values of  Merge_PDScoring_PDDriver to be more descriptive

PD_Scoring_Driver_Merge['Merge_PDScoring_PDDriver'] = PD_Scoring_Driver_Merge['Merge_PDScoring_PDDriver'].replace({'both': 'Both', 'left_only': 'PD Scoring Only', 'right_only': 'PD Driver Only'})

# Group by Merge_PDScoring_PDDriver, count the number of loans

print('Count of loans by Merge_PDScoring_PDDriver:')

PD_Scoring_Driver_Merge['Merge_PDScoring_PDDriver'].value_counts()


Count of loans by Merge_PDScoring_PDDriver:


Both               55629
PD Scoring Only        0
PD Driver Only         0
Name: Merge_PDScoring_PDDriver, dtype: int64

## Compare with PD WOE Data

In [44]:
# Merge PD_Data_Scored_red with PD_WOE_Prod_Data on Loan_Number

PD_Scoring_WOE_Merge = pd.merge(PD_Data_Scored_red, PD_WOE_Prod_Data[['Loan_Number']], on='Loan_Number', how='outer', indicator= 'Merge_PDScoring_PDWOE')

# Change values of  Merge_PDScoring_PDWOE to be more descriptive

PD_Scoring_WOE_Merge['Merge_PDScoring_PDWOE'] = PD_Scoring_WOE_Merge['Merge_PDScoring_PDWOE'].replace({'both': 'Both', 'left_only': 'PD Scoring Only', 'right_only': 'PD WOE Only'})

# Group by Merge_PDScoring_PDWOE and Corporate_Applicant_Ind, count the number of loans

print('Count of loans by Merge_PDScoring_PDWOE, Corporate_Applicant_Ind, and Sub_Product:')

print(PD_Scoring_WOE_Merge.groupby(['Merge_PDScoring_PDWOE', 'Corporate_Applicant_Ind','Sub_Product']).size().reset_index(name='Count').sort_values(by=['Merge_PDScoring_PDWOE', 'Corporate_Applicant_Ind']))

# Filter PD_Scoring_WOE_Merge to only include loans that are not in both PD_Data_Scored_red and PD_WOE_Prod_Data

PD_Scoring_WOE_Merge_not_in_both = PD_Scoring_WOE_Merge[PD_Scoring_WOE_Merge['Merge_PDScoring_PDWOE'] != 'Both']

# save PD_Scoring_WOE_Merge_not_in_both to csv

PD_Scoring_WOE_Merge_not_in_both[['Loan_Number','Sub_Product','Corporate_Applicant_Ind','Merge_PDScoring_PDWOE']].to_csv(Output_dir + '\\' + 'PD_Scoring_WOE_Merge_not_in_both.csv', index=False)

Count of loans by Merge_PDScoring_PDWOE, Corporate_Applicant_Ind, and Sub_Product:
   Merge_PDScoring_PDWOE  Corporate_Applicant_Ind     Sub_Product  Count
0        PD Scoring Only                      0.0             EQB    828
1        PD Scoring Only                      0.0  First National      0
2        PD Scoring Only                      0.0            GMAC      6
3        PD Scoring Only                      0.0        Paradigm      0
4        PD Scoring Only                      1.0             EQB   1423
5        PD Scoring Only                      1.0  First National      0
6        PD Scoring Only                      1.0            GMAC      0
7        PD Scoring Only                      1.0        Paradigm      0
8            PD WOE Only                      0.0             EQB      0
9            PD WOE Only                      0.0  First National      0
10           PD WOE Only                      0.0            GMAC      0
11           PD WOE Only                 

## Compare with Consolidated RESL Data

In [45]:
# Merge Consolidated_RESL_Data_Raw and PD_Data_Scored_red on Loan_Number

Consolidated_PD_Scoring = Consolidated_RESL_Data_Raw.merge(PD_Data_Scored_red[['Loan_Number']], how='outer', on = 'Loan_Number', suffixes=('_Consoldidated', '_Scoring'), indicator= '_merge_Consolidated_PD_Scoring') 

# change _merge_Consolidated_PD_Scoring to be more descriptive

Consolidated_PD_Scoring['_merge_Consolidated_PD_Scoring'] = Consolidated_PD_Scoring['_merge_Consolidated_PD_Scoring'].replace({'left_only': 'Consolidated_RESL_Only', 'right_only': 'PD_Scoring_Only', 'both': 'Both'}) 


# Group by _merge_Consolidated_PD_Scoring, count the number of loans

print('Count of loans by _merge_Consolidated_PD_Scoring:')

Consolidated_PD_Scoring['_merge_Consolidated_PD_Scoring'].value_counts()


Count of loans by _merge_Consolidated_PD_Scoring:


Both                      55629
Consolidated_RESL_Only     5741
PD_Scoring_Only               0
Name: _merge_Consolidated_PD_Scoring, dtype: int64

## Clean Data

In [46]:
# Keep  column Loan_Number, Corporate_Applicant_Ind

PD_Scored_Data = PD_Data_Scored_red[['Loan_Number','Corporate_Applicant_Ind']]

# Load LGD Scoring Data

## Extract Data

In [47]:
# add new SFR LGD model generated LGD and apply LGD floor 


if snapshot_num == 202212:

    LGD_Scoring_Data_Raw = pd.read_pickle(input_dir + '\\eqb_lgd_scored_2022_12 (from Abhi 20250410).pkl') # 20250406 George added

if snapshot_num == 202412:

    LGD_Scoring_Data_Raw = pd.read_pickle(input_dir + '\\eqb_lgd_scored_2024_12 (from Abhi 20250404).pkl') # 20250406 George added


## Sanity Check

In [48]:
# print columns in alphabetical order in the dataframe 

print(sorted(LGD_Scoring_Data_Raw.columns)) #20250402 George added

['Appraisal_Bin', 'Appraisal_Bin_WOE', 'Appraisal_Bin_WOE', 'BF_Appr_Prov_Dw', 'BF_LTV_Incl_Parter_Incl_HELOC_FSA_Dw', 'Default_Ind', 'Final_LGD', 'Foreclosure_Ind', 'Foreclosure_Ind_WOE', 'Insured_Ind', 'LR_Avg_LGD', 'LTV_Bin', 'LTV_Bin_WOE', 'LTV_Bin_WOE', 'LoanType', 'Loan_Number', 'MRS_Bin', 'Mapped_LGD', 'Occupancy', 'Occupancy_WOE', 'Pred_LGD', 'Province_Foreclosure', 'Province_Foreclosure_WOE', 'RemainingPrincipal_Excl_Partner', 'SL_Date', 'Sub_Product']


In [49]:
# Grouped by MRS_Bin, calculate the min and max of Mapped_LGD and Final_LGD

print('Grouped by MRS_Bin, calculate the min and max of Mapped_LGD and Final_LGD:')

print(LGD_Scoring_Data_Raw.groupby(['MRS_Bin']).agg({'Mapped_LGD': ['min', 'max'], 'Final_LGD': ['min', 'max']}).reset_index().sort_values(by=['MRS_Bin'])) 

Grouped by MRS_Bin, calculate the min and max of Mapped_LGD and Final_LGD:
  MRS_Bin Mapped_LGD           Final_LGD        
                 min       max       min     max
0       1   0.010912  0.010912    0.1074  0.1074
1       2   0.029987  0.029987    0.1821  0.1821
2       3   0.056927  0.056927    0.2528  0.2528
3       4   0.101136  0.101136    0.5355  0.5355


## Data Cleaning

In [50]:
# print duplicated columns 

print(LGD_Scoring_Data_Raw.columns[LGD_Scoring_Data_Raw.columns.duplicated()].tolist()) #20250412 George added

# Remove duplicated columns 

LGD_Scoring_Data = LGD_Scoring_Data_Raw.loc[:,~LGD_Scoring_Data_Raw.columns.duplicated()] #20250412 George added

['LTV_Bin_WOE', 'Appraisal_Bin_WOE']


In [51]:
# Rename Columns

LGD_Scoring_Data = LGD_Scoring_Data.rename({'MRS_Bin':'MRS_Bin_LGD','Final_LGD':'Model_LGD'}, axis = 'columns')

# Load LGD WOE Data

## Extract from SQL

In [52]:
# retrieve LGD WOE data from Production

def sql_LGD_WOE( Max_Feed_ID ):
    
    table = 'ET_Finance_Production.dbo.tb_X_WoE_LGD_Result' # 20250410 George added

    if pd.to_datetime(snapshot_date) <= pd.to_datetime('2024-10-31'):
        table = table + '_G2'  # 20250410 George added for Generation 2 model

    sql = f'''
      SELECT 
      [LoanNumber]
      ,[X1_Region_WoE]
      ,[X2_LTV_WoE]
      ,[X3_Appraisal_WoE]
      ,[X4_Occupancy_WoE]
      ,[LogOdds]
      ,[LGDResult]
      ,[CalibratedLGD]
      ,[RiskRating]
      ,[RiskRatingCategory]
      ,[SuccessOrFail]
      ,[Comments]
      ,[X_LGD_Overall]
      ,[Add_on_LGD]
      ,[Downturn_LGD]
      ,[Final_LGD]
       FROM {table}
       where RunID =  {Max_Feed_ID} 
            '''  
    return sql

LGD_WOE_Prod_Data_Raw = tool.download_from_sql( Server_Finance, Database_Production, sql_LGD_WOE(Max_Feed_ID) )

c:\Users\georgez\OneDrive - Equitable Bank\MD_RWA_Impact_Analysis (Joseph)\SFR\code\SFR_PD_Recalibration_2023_Lib.py:1732: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn,**kwargs)


## Sanity Check

In [53]:
# Count values of CalibratedLGD

print('Count of values of CalibratedLGD:')

LGD_WOE_Prod_Data_Raw['CalibratedLGD'].value_counts()

# Count values of RiskRating

print('Count of values of RiskRating:')

LGD_WOE_Prod_Data_Raw['RiskRating'].value_counts()

Count of values of CalibratedLGD:
Count of values of RiskRating:


Pool 2    20716
Pool 1    20384
Pool 4     6857
Pool 3     5445
Name: RiskRating, dtype: int64

## Data Cleaning

In [54]:
# Keep Columns LoanNumber, Calibrated LGD, X1_Region_WoE, X2_LTV_WoE, X3_Appraisal_WoE, X4_Occupancy_WoE, RiskRating

LGD_WOE_Prod_Data = LGD_WOE_Prod_Data_Raw[['LoanNumber','CalibratedLGD','X1_Region_WoE','X2_LTV_WoE','X3_Appraisal_WoE','X4_Occupancy_WoE','RiskRating']]


In [55]:

# Rename Columns: LoanNumber to Loan_Number, CalibratedLGD to Model_LGD, RiskRating to MRS_Bin_LGD, X1_Region_WoE to Province_Foreclosure_WOE, X2_LTV_WoE to LTV_Bin_WOE, X3_Appraisal_WoE to Appraisal_Bin_WOE, X4_Occupancy_WoE to Occupancy_WOE

LGD_WOE_Prod_Data = LGD_WOE_Prod_Data.rename(columns={'LoanNumber':'Loan_Number','CalibratedLGD':'Model_LGD','RiskRating':'MRS_Bin_LGD','X1_Region_WoE':'Province_Foreclosure_WOE','X2_LTV_WoE':'LTV_Bin_WOE','X3_Appraisal_WoE':'Appraisal_Bin_WOE','X4_Occupancy_WoE':'Occupancy_WOE'})


In [56]:

# Hard code Mapped LGD values because it is not in the data set. If Model_LGD = 0.1074, then Mapped_LGD = 0.010912; if Model_LGD = 0.1821, then Mapped_LGD = 0.029987; if Model_LGD = 0.2528, then Mapped_LGD = 0.056927; if Model_LGD = 0.5355, then Mapped_LGD = 0.101136. This is based on the mapping in the Abhi's scoring LGD data in Dec. 2024.

LGD_WOE_Prod_Data['Mapped_LGD'] = np.where(np.round(LGD_WOE_Prod_Data['Model_LGD'],4) == 0.1074, 0.010912, np.where(np.round(LGD_WOE_Prod_Data['Model_LGD'],4) == 0.1821, 0.029987, np.where(np.round(LGD_WOE_Prod_Data['Model_LGD'],4) == 0.2528, 0.056927, np.where(np.round(LGD_WOE_Prod_Data['Model_LGD'],4) == 0.5355, 0.101136, np.nan))))  #unfortunately this is hard-coded due to data availability, but the good thing is that if the numerical model_LGD values change due to model change, then the map results in missing values - which is a good thing to be easily noticed. If hard-coded based on MRS_Bin, then it is not easy to be noticed and quite dangerous to assign outdated values. SO this is a safe hard-coding!                


In [75]:
# Change Loan_Number to integer

LGD_WOE_Prod_Data['Loan_Number'] = LGD_WOE_Prod_Data['Loan_Number'].astype(int)

## Double-check

In [57]:
# Group by MRS_Bin_LGD, calculate the min and max of Mapped_LGD and Model_LGD

print('Grouped by MRS_Bin_LGD, calculate the min and max of Mapped_LGD and Model_LGD:')

print(LGD_WOE_Prod_Data.groupby(['MRS_Bin_LGD']).agg({'Mapped_LGD': ['min', 'max'], 'Model_LGD': ['min', 'max']}).reset_index().sort_values(by=['MRS_Bin_LGD'])) 

Grouped by MRS_Bin_LGD, calculate the min and max of Mapped_LGD and Model_LGD:
  MRS_Bin_LGD Mapped_LGD           Model_LGD          
                     min       max       min       max
0      Pool 1   0.010912  0.010912  0.107376  0.107376
1      Pool 2   0.029987  0.029987  0.182108  0.182108
2      Pool 3   0.056927  0.056927  0.252839  0.252839
3      Pool 4   0.101136  0.101136  0.535503  0.535503


# Merge All Loaded to Get PD Data

In [58]:
#  use reduce from functools to merge the following four data sets together on Loan_Number: PD WOE Data with Risk Rating Attribute Data, Consolidated_RESL_Data and PD Scored Data

from functools import reduce

new_pd_data_red = reduce(lambda left, right: pd.merge(left, right, on='Loan_Number', how='left', validate= '1:1'), [PD_WOE_Prod_Data, Risk_Rating_Attribute_Prod_Data, Consolidated_RESL_Data, PD_Scored_Data]) 




In [59]:
# print columns in alphabetical order in the new_pd_data_red data

print(sorted(new_pd_data_red.columns)) #20250402 George added


['Alt_Prime_Indicator', 'BF_Appr_Prov_Dw', 'BF_LTV_Incl_Parter_Incl_HELOC_FSA_Dw', 'BF_LTV_Tot_Exp_FSA_Dw_WF', 'Beacon_Avg_App_CoApp', 'Beacon_Avg_App_CoApp_WOE', 'Combo_LTV_Insured_Ind_WOE', 'Combo_Province_Metro_Override_WOE', 'Corporate_Applicant_Ind', 'Current_Delinquency_Days', 'Default_Indicator', 'Insured_Ind', 'Loan_Number', 'MRS_Bin_PD', 'Occupancy', 'PD_Post_MOC', 'Prior_24_Worse_Delinquent_Status_FMT_Adj', 'Prior_24_Worse_Delinquent_Status_FMT_Adj_WOE', 'RemainingPrincipal_Excl_Partner', 'RemainingPrincipal_Tot_Exp', 'SL_Date', 'Sub_Product']


In [60]:
# count values of Insured_Ind. !!!IMportant: Insured_Ind must have values "Not Insured" (not "Uninsured") to be consistent with model development (scoring will assume these contents - may be OK for now, but safer to be consistent with model development. In contrast, Insured_Class shoudl have values "Uninsured" (not "Not Insured") because that is assumed in the RWA code

print('Count of values of Insured_Ind:')

print(new_pd_data_red['Insured_Ind'].value_counts()) 

Count of values of Insured_Ind:
Not Insured    28922
Insured        24480
Name: Insured_Ind, dtype: int64


In [61]:
# For columns PD_Post_MOC_Pre_Adj and PD_Pre_MOC, if not in the data set, add them with nan #20250402 George added

if 'PD_Post_MOC_Pre_Adj' not in new_pd_data_red.columns: #20250402 George added

    new_pd_data_red['PD_Post_MOC_Pre_Adj'] = np.nan #20250402 George added

if 'PD_Pre_MOC' not in new_pd_data_red.columns: #20250402 George added

    new_pd_data_red['PD_Pre_MOC'] = np.nan #20250402 George added


# Rename AIRB_PD_Risk_Rating to be MRS_Bin #20250402 George added

new_pd_data_red = new_pd_data_red.rename({'AIRB_PD_Risk_Rating':'MRS_Bin'}, axis='columns')  #20250402 George added

In [62]:
# Group by MRS_Bin_PD, count distinct Loan_Number and min and max of PD_Post_MOC #20250402 George added
print('***************Checking Default Loans *****************') #20250402 George added
print(new_pd_data_red.groupby(['MRS_Bin_PD'], as_index=False).agg({'Loan_Number': pd.Series.nunique, 'PD_Post_MOC': [min, max]})) #20250402 George added



***************Checking Default Loans *****************


   MRS_Bin_PD Loan_Number PD_Post_MOC          
                  nunique         min       max
0     Default         368    1.000000  1.000000
1  Delinquent         372    0.629118  0.629118
2      Pool 1       26281    0.002228  0.002228
3      Pool 2       13784    0.003622  0.003622
4      Pool 3        5271    0.007377  0.007377
5      Pool 4        3037    0.011499  0.011499
6      Pool 5        1545    0.019067  0.019067
7      Pool 6        1181    0.034554  0.034554
8      Pool 7        1563    0.107857  0.107857


In [63]:
# # Remove Dafault Loans

# # Remove rows with PD_Post_MOC <=0 or >=1 (defaults) #20250402 George added

# new_pd_data_red = new_pd_data_red.loc[(new_pd_data_red['PD_Post_MOC'] > 0) & (new_pd_data_red['PD_Post_MOC'] < 1)] #20250402 George added

# # Group by MRS_Bin, count distinct Loan_Number and min and max of PD_Post_MOC #20250402 George added

# print('***************After Removing Default Loans*****************') #20250402 George added

# print(new_pd_data_red.groupby(['MRS_Bin'], as_index=False).agg({'Loan_Number': pd.Series.nunique, 'PD_Post_MOC': [min, max]})) #20250402 George added

In [64]:
# Group by Sub_Product, count distinct Loan_Number 

print('***************Checking Sub_Product*****************') #20250402 George added

print(new_pd_data_red.groupby(['Sub_Product'], as_index=False).agg({'Loan_Number': pd.Series.nunique})) 

***************Checking Sub_Product*****************
      Sub_Product  Loan_Number
0             EQB        43206
1  First National         7162
2        Paradigm         3034


# Merge All Loaded to Get LGD Data

In [92]:
# Set the LGD data

new_lgd_data = LGD_WOE_Prod_Data.copy()

In [93]:
# Add columns LGD_DT_Adjusted, Segment_Avg_LGD and Base_Line_LGD if they are not in the data set #20250412 George added

if 'LGD_DT_Adjusted' not in new_lgd_data.columns: # 20250412 George added
    new_lgd_data['LGD_DT_Adjusted'] = np.nan  # 20250412 George added, the numbers are not the latest and this will not be used in the final result

if 'Segment_Avg_LGD' not in new_lgd_data.columns: # 20250412 George added, same treatment as RWA for Concentra
    new_lgd_data['Segment_Avg_LGD'] = new_lgd_data.loc[:,'Mapped_LGD']

if 'Base_Line_LGD' not in new_lgd_data.columns: # 20250412 George added, same treatment as RWA for Concentra
    new_lgd_data['Base_Line_LGD'] = new_lgd_data.loc[:,'Mapped_LGD']

# Merge RWA Data Info with PD Data

In [94]:
#test = new_pd_data_red[ ['PD_Pre_MOC','PD_Post_MOC_Pre_Adj','PD_Post_MOC'] ].drop_duplicates()

rwa_raw_data_1 = pd.merge(rwa_raw_data_0, new_pd_data_red[PD_Cols_to_Keep], how='outer',on=['Loan_Number'], indicator= 'merge_Chenxi_PD')

rwa_raw_data_1['merge_Chenxi_PD'] = rwa_raw_data_1['merge_Chenxi_PD'].replace({'left_only': 'Loan in Chenxi only', 'right_only': 'Loan in PD Data only', 'both': 'Loan in both Chenxi and PD'}) #20250403 George added

# rwa_raw_data_2 =  rwa_raw_data_1.loc[~rwa_raw_data_1['PD_Pre_MOC'].isnull()] #remove records not existing in Chenxi's data since they cannot be compared

rwa_raw_data_2 = rwa_raw_data_1.copy()

rwa_raw_data_3 = rwa_raw_data_2.copy()



# Merge RWA info with LGD Data

In [95]:
rwa_raw_data_4 = pd.merge(rwa_raw_data_3,new_lgd_data[LGD_Cols_to_Keep], how='outer', on= ['Loan_Number'], indicator= 'merge_ChenxiPD_LGD')

rwa_raw_data_4['merge_ChenxiPD_LGD'] = rwa_raw_data_4['merge_ChenxiPD_LGD'].replace({'left_only': 'Loan in ChenxiPD only', 'right_only': 'Loan in LGD Data only', 'both': 'Loan in both ChenxiPD and LGD'})


# rwa_raw_data_4 = rwa_raw_data_4.rename({'MRS_Bin':'MRS_Bin_LGD','Final_LGD':'Model_LGD'}, axis = 'columns')

# Calculate RWA with 0 Addon

In [96]:
# Force Addon to be 0

rwa_raw_data_0Addon = rwa_raw_data_4.copy()
rwa_raw_data_0Addon['AddOn'] =0

In [97]:
print(CMHC_pd)

0.0001


In [98]:
# Run RWA calculation function

[df_out_0Addon, rwa_by_Insured_class_0Addon, rwa_by_MRS_Bin_0Addon, res_data_0Addon]  = Lib_RWA.rwa_calculation(df_input_data = rwa_raw_data_0Addon, lgd_gen_floor = lgd_gen_floor, CMHC_lgd = CMHC_lgd, CMHC_pd = CMHC_pd )



0
1
2


In [99]:
# print duplicated columns in res_data_0Addon - These are in LGD data set

print(res_data_0Addon.columns[res_data_0Addon.columns.duplicated()].tolist()) 


[]


In [100]:
# Save Data to parquet

res_data_0Addon.to_parquet(Output_dir +'\\'+'eqb_rwa_addon_zero.parquet', index=False)

# with pd.ExcelWriter(Output_dir +'\\'+'eqb_rwa_addon_zero.xlsx' )as writer:
#     res_data_0Addon.to_excel(writer, sheet_name='RWA_data', index=False)
#     df_out_0Addon.to_excel(writer, sheet_name="RWA") 
#     rwa_by_Insured_class_0Addon.to_excel(writer, sheet_name="rwa_by_Insured_class")
#     rwa_by_MRS_Bin_0Addon.to_excel(writer, sheet_name="rwa_by_MRS_Bin")

# Get LGD Addon and Calculate RWA

In [101]:
## retrieve DLGD floor readily available AddOn from Finance Production database
# Get max Feedid
Server_Finance = 'EQDWP01'  
Database_Production = 'ET_Finance_Production'   



def sql_max_FeedID( snapshot ):
    
    table = 'ET_Finance_Production.dbo.tb_RE_log' # 20250410 George added

    if pd.to_datetime(snapshot) <= pd.to_datetime('2024-10-31'):
        table = table + '_G2'  # 20250410 George added for Generation 2 model

    sql = f'''
      SELECT max([FeedID]) as max_FeedID
       FROM {table}
       where Reporting_date =  {snapshot} 
             
            '''  
    return sql

Max_Feed_ID= tool.download_from_sql( Server_Finance, Database_Production, sql_max_FeedID( "'" + snapshot_date + "'" ) )['max_FeedID'][0]

def sql_LGD_addon( Max_Feed_ID ):
    
    table = 'ET_Finance_Production.dbo.tb_X_WoE_LGD_Result' # 20250410 George added

    if pd.to_datetime(snapshot_date) <= pd.to_datetime('2024-10-31'):
        table = table + '_G2'  # 20250410 George added for Generation 2 model

    sql = f'''
      SELECT 
       [LoanNumber]
       ,[Add_on_LGD] as AddOn
       FROM {table}
       where RunID =  {Max_Feed_ID} 
            '''  
    return sql

Add_on_LGD= tool.download_from_sql( Server_Finance, Database_Production, sql_LGD_addon(Max_Feed_ID) )

Add_on_LGD['LoanNumber'] = Add_on_LGD['LoanNumber'].astype(str).astype(int)

rwa_raw_data_TrueAddon = pd.merge(rwa_raw_data_4, Add_on_LGD, how='left', left_on= ['Loan_Number'],right_on=['LoanNumber'])


#rwa_raw_data_5['Final_LGD'] = rwa_raw_data_5['Final_LGD_old']
#rwa_raw_data_5['PD_Post_MOC'] = rwa_raw_data_5['CalibratedPD_old']

c:\Users\georgez\OneDrive - Equitable Bank\MD_RWA_Impact_Analysis (Joseph)\SFR\code\SFR_PD_Recalibration_2023_Lib.py:1732: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn,**kwargs)
c:\Users\georgez\OneDrive - Equitable Bank\MD_RWA_Impact_Analysis (Joseph)\SFR\code\SFR_PD_Recalibration_2023_Lib.py:1732: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn,**kwargs)


In [102]:
# Run RWA calculation function

[df_out, rwa_by_Insured_class, rwa_by_MRS_Bin, res_data]  = Lib_RWA.rwa_calculation(rwa_raw_data_TrueAddon, lgd_gen_floor, CMHC_lgd, CMHC_pd)



0
1
2


# Sanity Check the Outputs

In [103]:
# print snapshot date 

print('Snapshot date: ' + snapshot_date) #20250403 George added

# print the shape of res_data

print('Shape of res_data: ' + str(res_data.shape)) #20250403 George added

# Group by Insured_Indicator, count number of Loan_Number, sum of EAD and RWA_new

print('***************Group by Insured_Indicator*****************') #20250403 George added

Sumry = res_data.groupby(['Insured_class'], as_index=False).agg({'Loan_Number': pd.Series.nunique, 'EAD': sum, 'RWA_new': sum}) #20250403 George added

# Add a total row to Sumry

Sumry = Sumry.append(Sumry.sum(numeric_only=True), ignore_index=True) #20250403 George added
Sumry.at[Sumry.index[-1], 'Insured_class'] = 'Total' #20250403 George added

# Add a column RWA_Ratio as the ratio of RWA_new and EAD

Sumry['RWA_Ratio'] = Sumry['RWA_new'] / Sumry['EAD'] #20250403 George added

# print the summary

print(Sumry) #20250403 George added

# print columns in alphabetical ored

print(sorted(res_data.columns)) #20250403 George added

Snapshot date: 2024-12-31
Shape of res_data: (53402, 50)
***************Group by Insured_Indicator*****************
  Insured_class  Loan_Number           EAD       RWA_new  RWA_Ratio
0            CG       4534.0  1.290269e+09  9.586197e+07   0.074296
1          CMHC      11040.0  3.061192e+09  5.133710e+07   0.016770
2         Sagen       8832.0  2.293171e+09  1.815925e+08   0.079188
3     Uninsured      28996.0  1.562863e+10  3.627522e+09   0.232107
4         Total      53402.0  2.227326e+10  3.956314e+09   0.177626
['AddOn', 'Advance_Amount', 'Alt_Prime_Indicator', 'Appraisal_Bin_WOE', 'BF_LTV_Tot_Exp_FSA_Dw_WF', 'Base_Line_LGD', 'Beacon_Avg_App_CoApp', 'Beacon_Avg_App_CoApp_WOE', 'CalibratedPD', 'Combo_LTV_Insured_Ind_WOE', 'Combo_Province_Metro_Override_WOE', 'Corporate_Applicant_Ind', 'DLGD_floor', 'EAD', 'EAD_DLGD_Modified', 'EAD_Post_CRM', 'Final_LGD', 'Insured_Ind', 'Insured_class', 'LGD_Before_Floors', 'LGD_DT_Adjusted', 'LTV_Bin_WOE', 'LoanNumber', 'Loan_Number', 'MRS_Bin_LG

C:\Users\georgez\AppData\Local\Temp\ipykernel_8452\1574102103.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Sumry = Sumry.append(Sumry.sum(numeric_only=True), ignore_index=True) #20250403 George added


# Save Data

In [104]:
# Save Data to Parquet file

res_data.to_parquet(Output_dir +'\\'+'eqb_rwa_original.parquet', index=False)

# with pd.ExcelWriter(Output_dir +'\\'+'eqb_rwa_original.xlsx' )as writer:
#     res_data.to_excel(writer, sheet_name='RWA_Data', index=False)
#     df_out.to_excel(writer, sheet_name="RWA") 
#     rwa_by_Insured_class.to_excel(writer, sheet_name="rwa_by_Insured_class")
#     rwa_by_MRS_Bin.to_excel(writer, sheet_name="rwa_by_MRS_Bin")